In [1]:
from nemo_text_processing.text_normalization.normalize import Normalizer

In [2]:
normaliser_sv = Normalizer(input_case='cased', lang='sv', cache_dir="/tmp/nemo_cache_sv", overwrite_cache=False, post_process=True)

In [3]:
from pathlib import Path
APIFILES = Path("/home/joregan/rdapi_subset_files")
OUTFILES = Path("/home/joregan/rdapi_subset_norm")

In [4]:
import json

In [5]:
!pip install beautifulsoup4

In [6]:
from bs4 import BeautifulSoup

In [7]:
def get_speaker_paragraphs(html):
    if "<p>" in html or "<P>" in html:
        soup = BeautifulSoup(html, 'html.parser')
        paragraphs = []
        for para in soup.find_all("p"):
            if para.text.strip() != "" and not para.text.strip().startswith("STYLEREF Kantrubrik"):
                paragraphs.append(para.text.strip())
        return paragraphs
    else:
        text = html.strip().replace("\r\n", "\n").replace("\r", "\n")
        return text.split("\n")

In [ ]:
for jsonfile in APIFILES.glob("*"):
    with open(str(jsonfile)) as apifile:
        print("Processing", str(jsonfile))
        rdfile = json.load(apifile)
        for viddata in rdfile['videodata']:
            for speaker in viddata['speakers']:
                raw_text = speaker['anftext']
                raw_text = raw_text.replace("½", "och halv")
                paras = get_speaker_paragraphs(raw_text)
                normed = normaliser_sv.normalize_list(paras)
                speaker['anftext'] = "\r\n".join(normed)
        stem = jsonfile.stem
        outname = str(OUTFILES / stem)
        with open(outname, "w") as outfile:
            json.dump(rdfile, outfile)